# Pip commands needed, when using collab

In [10]:
# !pip install ortools geopy
# !pip install openrouteservice
# !pip install requests
# !pip install polyline

# Imports and libraries

In [11]:
from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial import distance
import pandas as pd
import numpy as np
import random
import json
import requests
import polyline
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
from sklearn.metrics import silhouette_score

# Project Parameters

In [12]:
GRAPHHOPPER_API_KEY = '053930b2-f363-4354-ad4f-82126696ca9f'
NUM_CUSTOMERS = 1000

# --- DBSCAN Parameters ---
DBSCAN_EPS = 0.015
DBSCAN_MIN_SAMPLES = 18

# --- Zoning for the CORE customers ---
NUM_OPERATING_ZONES = 5

# --- Vehicle & Solver Constraints (remain the same) ---
VEHICLE_CAPACITY = 8
SERVICE_TIME_MINUTES = 5
AVG_SPEED_KMH = 35
MAX_DRIVERS_PER_ZONE = 20
SOLVER_TIME_LIMIT_SECONDS = 20

# Helper Functions

## Time functions

In [13]:
def time_to_minutes(t_str):
    if pd.isna(t_str): return 0
    t_str = str(t_str).strip()
    hour, minute = map(int, t_str.split(":"))
    return hour * 60 + minute

def minutes_to_time_str(minutes):
    if pd.isna(minutes): return ""
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"

## Data

In [14]:
def generate_dummy_data(n=1000):
    print(f"Generating dummy data for {n} customers...")
    depot = {"Customer Name": "Depot", "Latitude": 31.6, "Longitude": 35.15, "Time Window": "08:00–20:00"}
    customer_data = []
    for i in range(n):
        name = f"Customer_{i+1}"
        lat = np.random.uniform(31.5, 31.7)
        lon = np.random.uniform(35.0, 35.3)
        pref_start_hour = random.randint(10, 18)
        time_window = f"{pref_start_hour:02d}:00–{pref_start_hour+1:02d}:00"
        customer_data.append((name, lat, lon, time_window))
    df_customers = pd.DataFrame(customer_data, columns=["Customer Name", "Latitude", "Longitude", "Time Window"])
    df_depot = pd.DataFrame([depot])
    return pd.concat([df_depot, df_customers], ignore_index=True)

## Algorithm

In [15]:
def solve_cvrptw_for_zone(zone_df, depot_df):
    """
    Solves the Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)
    for a single geographic zone, allowing for multiple trips per driver.
    """
    # --- A. Data Model Creation ---
    data_df = pd.concat([depot_df, zone_df], ignore_index=True).reset_index(drop=True)

    data = {}
    data['locations'] = list(zip(data_df['Latitude'], data_df['Longitude']))
    data['names'] = data_df['Customer Name'].tolist()
    data['time_windows'] = [tuple(map(time_to_minutes, tw.split('–'))) for tw in data_df['Time Window']]
    data['demands'] = [0] + [1] * len(zone_df)
    data['num_vehicles'] = MAX_DRIVERS_PER_ZONE
    data['vehicle_capacities'] = [VEHICLE_CAPACITY] * MAX_DRIVERS_PER_ZONE
    data['depot'] = 0

    size = len(data['locations'])
    matrix = np.zeros((size, size), dtype=int)
    for from_idx in range(size):
        for to_idx in range(size):
            if from_idx == to_idx: continue
            dist_km = geodesic(data['locations'][from_idx], data['locations'][to_idx]).km
            matrix[from_idx][to_idx] = int((dist_km / AVG_SPEED_KMH) * 60)
    data['distance_matrix'] = matrix

    # --- B. Routing Model Setup ---
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)

    # --- C. Callbacks and Dimensions ---
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = data['distance_matrix'][from_node][to_node]
        service_time = SERVICE_TIME_MINUTES if from_node != data['depot'] else 0
        return service_time + travel_time
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index, 0, data['vehicle_capacities'], False, 'Capacity'
    )

    routing.AddDimension(
        transit_callback_index, 120, 24*60, False, 'Time'
    )
    time_dimension = routing.GetDimensionOrDie('Time')
    for location_idx, time_window in enumerate(data['time_windows']):
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    for i in range(data['num_vehicles']):
        index = routing.Start(i)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0], data['time_windows'][0][1])

    # --- D. Penalties and Costs ---
    for node_idx in range(1, len(data['locations'])):
        routing.AddDisjunction([manager.NodeToIndex(node_idx)], 999999)
    routing.SetFixedCostOfAllVehicles(10000)

    # --- E. Solve ---
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(SOLVER_TIME_LIMIT_SECONDS)
    solution = routing.SolveWithParameters(search_parameters)

    # --- F. Extract Solution ---
    if solution:
        zone_routes = []
        dropped_nodes = []
        for node in range(1, routing.nodes()):
            if routing.IsStart(node) or routing.IsEnd(node):
                 continue

            index = manager.NodeToIndex(node)
            if solution.Value(routing.NextVar(index)) == index:
                # manager.IndexToNode() converts the solver's internal index back to our data index
                dropped_nodes.append(data['names'][manager.IndexToNode(index)])

        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            if routing.IsEnd(solution.Value(routing.NextVar(index))):
                continue

            vehicle_route = []
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                time_var = time_dimension.CumulVar(index)
                vehicle_route.append({
                    "name": data['names'][node_index],
                    "time_minutes": solution.Min(time_var)
                })
                index = solution.Value(routing.NextVar(index))
            zone_routes.append(vehicle_route)
        return zone_routes, dropped_nodes
    else:
        return None, zone_df['Customer Name'].tolist()

# Exe

In [16]:
score=""

# --- STEP 1: LOAD DATA ---
# full_df = generate_dummy_data(n=NUM_CUSTOMERS)
full_df = pd.read_csv('/content/full_df.csv')
depot_df = full_df.iloc[:1].copy()
customers_df = full_df.iloc[1:].copy()

# --- STEP 2: DYNAMIC ZONING & OUTLIER HANDLING ---
print("\n--- Running 2-Stage Dynamic Zoning ---")
coords = customers_df[['Latitude', 'Longitude']].values

# STAGE 2A: Use DBSCAN to find noise/outliers
print(f"Stage 1: Identifying noise with DBSCAN...")
db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES).fit(coords)
customers_df['dbscan_cluster'] = db.labels_

# STAGE 2B: Separate core customers from noise customers
outlier_customers_df = customers_df[customers_df['dbscan_cluster'] == -1].copy()
core_customers_df = customers_df[customers_df['dbscan_cluster'] != -1].copy()
print(f"-> Found {len(core_customers_df)} 'core' customers and {len(outlier_customers_df)} 'outlier' customers.")

# STAGE 2C: Use K-Means to create balanced zones from the core customers
if not core_customers_df.empty:
    print(f"Stage 2: Partitioning core customers into {NUM_OPERATING_ZONES} balanced zones...")
    core_coords = core_customers_df[['Latitude', 'Longitude']].values
    kmeans = KMeans(n_clusters=NUM_OPERATING_ZONES, random_state=42, n_init='auto').fit(core_coords)
    core_customers_df.loc[:, 'Zone'] = kmeans.labels_

    if len(core_customers_df['Zone'].unique()) > 1: # Silhouette score needs at least 2 clusters
        score = silhouette_score(core_coords, core_customers_df['Zone'])
        print(f"-> Zoning Quality (Silhouette Score): {score:.3f} (closer to 1 is better)")
    else:
        print("-> Zoning Quality (Silhouette Score): Not applicable (only one zone).")

    # STAGE 2D: Absorb outliers into the nearest core zone
    if not outlier_customers_df.empty:
        print("Stage 3: Assigning outlier customers to the nearest core zone...")
        zone_centroids = core_customers_df.groupby('Zone')[['Latitude', 'Longitude']].mean().to_numpy()
        outlier_coords = outlier_customers_df[['Latitude', 'Longitude']].to_numpy()

        # Find the closest zone for each outlier
        closest_zone_indices = [np.argmin(distance.cdist([outlier], zone_centroids)) for outlier in outlier_coords]
        outlier_customers_df.loc[:, 'Zone'] = closest_zone_indices

        # Combine the core and newly-assigned outlier customers
        customers_df = pd.concat([core_customers_df, outlier_customers_df])
        print(f"-> All {len(outlier_customers_df)} outlier customers have been assigned a zone.")
    else:
        customers_df = core_customers_df
else: # Case where DBSCAN finds no core clusters
    print("Warning: No core clusters found. Treating all customers as a single zone.")
    customers_df.loc[:, 'Zone'] = 0

# Flag outliers for special handling later
customers_df['is_outlier'] = customers_df['dbscan_cluster'] == -1

# --- STEP 3: SOLVE FOR EACH ZONE & STORE RESULTS ---
final_results_list = []
all_routes_for_visualization = []
all_dropped_customers = [] # Customers the solver fails on
global_driver_id_counter = 0

for zone_id, zone_df in customers_df.groupby('Zone'):
    # We only run the solver on the non-outlier customers
    core_zone_df = zone_df[zone_df['is_outlier'] == False]

    print(f"\n{'='*20} Solving for Zone {zone_id} ({len(core_zone_df)} core customers) {'='*20}")

    routes, dropped_nodes = solve_cvrptw_for_zone(core_zone_df, depot_df)

    if routes:
        print(f"Solution found for Zone {zone_id}. Minimum drivers required: {len(routes)}")
        # In STEP 3, find this loop: for zone_id, zone_df in customers_df.groupby('Zone'):

        zone_driver_manifests = []

        for driver_route in routes:
            driver_id = global_driver_id_counter

            driver_manifest = {
                "driver_id": driver_id,
                "zone_id": zone_id,
                "total_deliveries": 0,
                "trips": []
            }

            # Deconstruct the full-day route into individual trips
            trips = []
            current_trip = []
            for i, stop in enumerate(driver_route):
                current_trip.append(stop)
                if stop['name'] == 'Depot' and i < len(driver_route) - 1:
                    trips.append(current_trip)
                    current_trip = [] # Start a new trip
            if current_trip:
                last_stop_node = driver_route[-1]
                if last_stop_node['name'] != 'Depot':
                    last_stop_node = {'name': 'Depot', 'time_minutes': driver_route[-1]['time_minutes'] + 30}
                current_trip.append(last_stop_node)
                trips.append(current_trip)

            # Process each trip for the manifest
            for i, trip in enumerate(trips):
                trip_data = {
                    "trip_number": i + 1,
                    "depot_departure_time": minutes_to_time_str(trip[0]['time_minutes']),
                    "stops": []
                }

                # The actual delivery stops for this trip
                delivery_stops = trip[1:-1] # Exclude start and end depot stops
                for stop_order, stop in enumerate(delivery_stops):
                    customer_info = customers_df.loc[customers_df['Customer Name'] == stop['name']].iloc[0]
                    stop_details = {
                        "stop_order": stop_order + 1,
                        "customer_name": stop['name'],
                        "latitude": customer_info['Latitude'],
                        "longitude": customer_info['Longitude'],
                        "delivery_time": minutes_to_time_str(stop['time_minutes']),
                        "time_window": customer_info['Time Window']
                    }
                    trip_data["stops"].append(stop_details)

                if trip_data["stops"]: # Only add trip if it has deliveries
                    driver_manifest["trips"].append(trip_data)
                    driver_manifest["total_deliveries"] += len(trip_data["stops"])

            zone_driver_manifests.append(driver_manifest)

            # --- This part for visualization remains the same ---
            route_for_viz = {'driver_id': driver_id, 'zone_id': zone_id, 'path_coords': [], 'stops': []}
            full_path_coords = []
            customer_stops_for_viz = []
            for stop in driver_route:
                loc_df = depot_df if stop['name'] == 'Depot' else customers_df[customers_df['Customer Name'] == stop['name']]
                lon, lat = loc_df[['Longitude', 'Latitude']].iloc[0]
                full_path_coords.append([lon, lat])
                if stop['name'] != 'Depot':
                    arrival_time = minutes_to_time_str(stop['time_minutes'])
                    customer_stops_for_viz.append({'name': stop['name'], 'lat': lat, 'lon': lon, 'arrival': arrival_time})
            route_for_viz['path_coords'] = full_path_coords
            route_for_viz['stops'] = customer_stops_for_viz
            all_routes_for_visualization.append(route_for_viz)
            # --- End of visualization part ---

            global_driver_id_counter += 1

        # Add the collected manifest data to our final list
        final_results_list.extend(zone_driver_manifests)

        if dropped_nodes:
            all_dropped_customers.extend(dropped_nodes)
    else:
        all_dropped_customers.extend(core_zone_df['Customer Name'].tolist())


# --- STEP 4: FINAL SUMMARY ---
print(f"\n\n{'='*20} FINAL SUMMARY {'='*20}")
print(f"Total Drivers Required for Core Routes: {global_driver_id_counter}")
total_customers = len(customers_df)
# Total served = core customers - solver drops
served_core_count = len(core_customers_df) - len(all_dropped_customers)
print(f"Optimized Deliveries (Core): {served_core_count} / {len(core_customers_df)}")
print(f"Outlier Deliveries (to be handled manually): {len(outlier_customers_df)}")
print("-" * 57)
unserved_count = len(all_dropped_customers)
print(f"Total Customers Unserved by Solver: {unserved_count}")
print(f"{'='*57}")

# --- STEP 5: VISUALIZE RESULTS ---
print("\n\n--- Generating Route Visualization Map ---")

depot_location = [depot_df['Latitude'].iloc[0], depot_df['Longitude'].iloc[0]]
m = folium.Map(location=depot_location, zoom_start=11, tiles='CartoDB positron')

folium.Marker(location=depot_location, popup="<strong>Depot</strong>", icon=folium.Icon(color="red", icon="industry", prefix='fa')).add_to(m)

zone_colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue']

if GRAPHHOPPER_API_KEY != '053930b2-f363-4354-ad4f-82126696ca9f' and GRAPHHOPPER_API_KEY:
    for route_data in all_routes_for_visualization:
        color = zone_colors[route_data['zone_id'] % len(zone_colors)]

        # ... (GraphHopper API call remains the same) ...
        api_url = 'https://graphhopper.com/api/1/route'
        points_for_api = [f"{coord[1]},{coord[0]}" for coord in route_data['path_coords']]
        params = {'key': GRAPHHOPPER_API_KEY, 'vehicle': 'car', 'points_encoded': 'true', 'instructions': 'false', 'point': points_for_api}
        try:
            response = requests.get(api_url, params=params)
            if response.status_code == 200:
                encoded_polyline = response.json()['paths'][0]['points']
                decoded_points = polyline.decode(encoded_polyline)
                folium.PolyLine(locations=decoded_points, color=color, weight=3, opacity=0.7, popup=f"Driver {route_data['driver_id']} (Zone {route_data['zone_id']})").add_to(m)
        except Exception: pass

# Mark the different types of customers on the map
for _, customer in customers_df.iterrows():
    color = zone_colors[customer['Zone'] % len(zone_colors)]

    if customer['is_outlier']:
        # Outliers get a distinct icon but colored by their assigned zone
        folium.Marker(
            location=[customer['Latitude'], customer['Longitude']],
            popup=f"<b>OUTLIER: {customer['Customer Name']}</b><br>Assigned to Zone {customer['Zone']}",
            icon=folium.Icon(color=color, icon='exclamation-triangle', prefix='fa')
        ).add_to(m)
    elif customer['Customer Name'] in all_dropped_customers:
        # Core customers dropped by the solver
          folium.Marker(
            location=[customer['Latitude'], customer['Longitude']],
            popup=f"<b>SOLVER-DROPPED: {customer['Customer Name']}</b><br>Zone {customer['Zone']}",
            icon=folium.Icon(color='black', icon='times-circle', prefix='fa')
        ).add_to(m)
    else:
        # Successfully routed core customers
        folium.Marker(
            location=[customer['Latitude'], customer['Longitude']],
            popup=f"<b>{customer['Customer Name']}</b><br>Zone {customer['Zone']}",
            icon=folium.Icon(color=color, icon='circle', prefix='fa')
        ).add_to(m)



--- Running 2-Stage Dynamic Zoning ---
Stage 1: Identifying noise with DBSCAN...
-> Found 226 'core' customers and 774 'outlier' customers.
Stage 2: Partitioning core customers into 5 balanced zones...
-> Zoning Quality (Silhouette Score): 0.540 (closer to 1 is better)
Stage 3: Assigning outlier customers to the nearest core zone...
-> All 774 outlier customers have been assigned a zone.

==================== Solving for Zone 0 (52 core customers) ====================
Solution found for Zone 0. Minimum drivers required: 7

==================== Solving for Zone 1 (27 core customers) ====================
Solution found for Zone 1. Minimum drivers required: 4

==================== Solving for Zone 2 (86 core customers) ====================
Solution found for Zone 2. Minimum drivers required: 11

==================== Solving for Zone 3 (34 core customers) ====================
Solution found for Zone 3. Minimum drivers required: 5

==================== Solving for Zone 4 (27 core customers

# Output

## Map and export

In [17]:
print("\n\n--- Generating Visualization Maps ---")

depot_location = [depot_df['Latitude'].iloc[0], depot_df['Longitude'].iloc[0]]
zone_colors = ['blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue']

# --- MAP 1: Strategic Zone Map ---
print("-> Generating Map 1: Strategic Zone Overview...")
zone_map = folium.Map(location=depot_location, zoom_start=11, tiles='CartoDB positron')

folium.Marker(
    location=depot_location,
    popup="<strong>Depot</strong>",
    icon=folium.Icon(color="red", icon="industry", prefix='fa')
).add_to(zone_map)

# Add all customers to the zone map, colored by their assigned zone
for _, customer in customers_df.iterrows():
    color = zone_colors[customer['Zone'] % len(zone_colors)]

    if customer['is_outlier']:
        # Outliers get a distinct icon to stand out
        icon_type = 'exclamation-triangle'
        popup_text = f"<b>OUTLIER: {customer['Customer Name']}</b><br>Assigned to Zone {customer['Zone']}"
    else:
        # Core customers get a standard circle
        icon_type = 'circle'
        popup_text = f"<b>{customer['Customer Name']}</b><br>Zone {customer['Zone']}"

    folium.Marker(
        location=[customer['Latitude'], customer['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color=color, icon=icon_type, prefix='fa')
    ).add_to(zone_map)

# Save the strategic zone map
zone_map_filename = "1_strategic_zone_map.html"
zone_map.save(zone_map_filename)
print(f"✅ Strategic Zone Map saved to: '{zone_map_filename}'")


# --- MAP 2: Operational Route Map ---
print("-> Generating Map 2: Detailed Driver Routes...")
route_map = folium.Map(location=depot_location, zoom_start=11, tiles='CartoDB positron')

folium.Marker(
    location=depot_location,
    popup="<strong>Depot</strong>",
    icon=folium.Icon(color="red", icon="industry", prefix='fa')
).add_to(route_map)

# Draw the optimized routes for each driver
if GRAPHHOPPER_API_KEY != 'YOUR_GRAPHHOPPER_API_KEY_HERE' and GRAPHHOPPER_API_KEY:
    for route_data in all_routes_for_visualization:
        color = zone_colors[route_data['zone_id'] % len(zone_colors)]

        # Get road path from GraphHopper
        api_url = 'https://graphhopper.com/api/1/route'
        points_for_api = [f"{coord[1]},{coord[0]}" for coord in route_data['path_coords']]
        params = {'key': GRAPHHOPPER_API_KEY, 'vehicle': 'car', 'points_encoded': 'true', 'instructions': 'false', 'point': points_for_api}

        try:
            response = requests.get(api_url, params=params)
            if response.status_code == 200:
                encoded_polyline = response.json()['paths'][0]['points']
                decoded_points = polyline.decode(encoded_polyline)
                folium.PolyLine(
                    locations=decoded_points,
                    color=color,
                    weight=3,
                    opacity=0.7,
                    popup=f"Driver {route_data['driver_id']} (Zone {route_data['zone_id']})"
                ).add_to(route_map)
        except Exception as e:
            print(f"  - Warning: Could not fetch route from API for Driver {route_data['driver_id']}. Error: {e}")

# Add markers for only the successfully routed (core) customer stops
for route_data in all_routes_for_visualization:
    color = zone_colors[route_data['zone_id'] % len(zone_colors)]
    for stop in route_data['stops']:
         folium.Marker(
             location=[stop['lat'], stop['lon']],
             popup=f"<b>{stop['name']}</b><br>Driver {route_data['driver_id']}<br>Arrival: {stop['arrival']}",
             icon=folium.Icon(color=color, icon='circle', prefix='fa')
         ).add_to(route_map)

# Also add the outlier and dropped customers to the route map for context
# Outliers (assigned but not routed)
outliers_on_route_map = customers_df[customers_df['is_outlier']]
for _, customer in outliers_on_route_map.iterrows():
    color = zone_colors[customer['Zone'] % len(zone_colors)]
    folium.Marker(
        location=[customer['Latitude'], customer['Longitude']],
        popup=f"<b>OUTLIER: {customer['Customer Name']}</b><br>Assigned to Zone {customer['Zone']}",
        icon=folium.Icon(color=color, icon='exclamation-triangle', prefix='fa')
    ).add_to(route_map)

# Dropped by solver
solver_dropped_df = customers_df[customers_df['Customer Name'].isin(all_dropped_customers)]
for _, customer in solver_dropped_df.iterrows():
    folium.Marker(
        location=[customer['Latitude'], customer['Longitude']],
        popup=f"<b>SOLVER-DROPPED: {customer['Customer Name']}</b>",
        icon=folium.Icon(color='black', icon='times-circle', prefix='fa')
    ).add_to(route_map)

# Save the operational route map
route_map_filename = "2_operational_route_map.html"
route_map.save(route_map_filename)
print(f"✅ Operational Route Map saved to: '{route_map_filename}'")


# Display one of the maps in the notebook output (e.g., the detailed route map)
print("\nDisplaying the detailed operational route map below...")
route_map



--- Generating Visualization Maps ---
-> Generating Map 1: Strategic Zone Overview...
✅ Strategic Zone Map saved to: '1_strategic_zone_map.html'
-> Generating Map 2: Detailed Driver Routes...
✅ Operational Route Map saved to: '2_operational_route_map.html'

Displaying the detailed operational route map below...


In [18]:

# Save and display the map
output_filename = "dynamic_zoning_map.html"
m.save(output_filename)
print(f"\n✅ Interactive map has been saved to: '{output_filename}'")


✅ Interactive map has been saved to: 'dynamic_zoning_map.html'


In [19]:
# ==============================================================================
# --- STEP 6: EXPORT DATA TO JSON FILES ---
# ==============================================================================
print("\n--- Exporting data to JSON files ---")

# --- FILE 1: The Detailed Delivery Manifest ---
manifest_filename = "delivery_manifest.json"
with open(manifest_filename, 'w') as f:
    json.dump(final_results_list, f, indent=2)
print(f"✅ Detailed driver manifest saved to: '{manifest_filename}'")

# --- FILE 2: The High-Level Summary Report ---
total_drivers = len(final_results_list)
total_trips = sum(len(driver['trips']) for driver in final_results_list)
total_core_deliveries = sum(driver['total_deliveries'] for driver in final_results_list)
avg_trips_per_driver = total_trips / total_drivers if total_drivers > 0 else 0
avg_deliveries_per_driver = total_core_deliveries / total_drivers if total_drivers > 0 else 0

summary_data = {
    "total_drivers_required": total_drivers,
    "total_customers": len(customers_df),
    "customers": {
        "count": len(outlier_customers_df),
        "late": total_core_deliveries,
        "dropped_by_solver": len(all_dropped_customers)
    },
    "performance_metrics": {
        "total_trips_executed": total_trips,
        "average_trips_per_driver": round(avg_trips_per_driver, 2),
        "average_deliveries_per_driver": round(avg_deliveries_per_driver, 2),
        "silhouette_score": score
    }
}

summary_filename = "summary_report.json"
with open(summary_filename, 'w') as f:
    json.dump(summary_data, f, indent=2)
print(f"✅ High-level summary report saved to: '{summary_filename}'")


--- Exporting data to JSON files ---
✅ Detailed driver manifest saved to: 'delivery_manifest.json'
✅ High-level summary report saved to: 'summary_report.json'
